# 转换

在这个笔记本中，我们将探讨如何使用大型语言模型进行文本转换任务，例如语言翻译、拼写和语法检查、语气调整和格式转换。

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 读取本地的 .env 文件

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 这是模型输出的随机程度
    )
    return response.choices[0].message["content"]

## 翻译

ChatGPT训练了许多语言的来源。这使得模型具有翻译能力。以下是如何使用此功能的一些示例。

In [4]:
prompt = f"""
将以下英文文本翻译成中文：\
```Hi, I would like to order a blender```\
"""
response = get_completion(prompt)
print(response)


嗨，我想订购一个搅拌机。


In [6]:
prompt = f"""
告诉我这是哪种语言：
```我希望我的“自我”永远“滋滋”地响，翻腾不休，\
就像火炭上的一滴糖 - 王小波```
"""
response = get_completion(prompt)
print(response)

这是中文。


In [7]:
prompt = f"""
将以下文本翻译成中文、西班牙语和英语海盗：\
```我想订购一个篮球```\
"""
response = get_completion(prompt)
print(response)

中文海盗：吾欲订购一枚篮球
西班牙语海盗：Quiero ordenar una pelota de baloncesto
英语海盗：I be wantin' to order a basketball


In [8]:
prompt = f"""
请用正式和非正式的形式将以下文本翻译成中文：
“Would you like to order a pillow?”
"""
response = get_completion(prompt)
print(response)

正式：您是否需要订购一个枕头？
非正式：你要不要订一个枕头？


### 通用翻译器
想象一下，你是一家大型跨国电商公司的IT负责人。用户用他们的母语向你发送IT问题。你的员工来自世界各地，只会说他们的母语。你需要一个通用翻译器！

In [9]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [10]:
for issue in user_messages:
    prompt = f"告诉我这是什么语言：```{issue}```"
    lang = get_completion(prompt)
    print(f"原始消息（{lang}）：{issue}")

    prompt = f"""
    将以下文本翻译成英文和中文：```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

原始消息（这是法语。）：La performance du système est plus lente que d'habitude.
英文翻译：The system performance is slower than usual.

中文翻译：系统性能比平常慢。 

原始消息（这是西班牙语。）：Mi monitor tiene píxeles que no se iluminan.
英文翻译：My monitor has pixels that don't light up.

中文翻译：我的显示器有一些像素不亮。 

原始消息（这是意大利语。）：Il mio mouse non funziona
英文翻译：My mouse is not working.

中文翻译：我的鼠标不工作。 

原始消息（这是波兰语。）：Mój klawisz Ctrl jest zepsuty
英文：My Ctrl key is broken
中文：我的Ctrl键坏了 

原始消息（这是中文。）：我的屏幕在闪烁
英文：My screen is flickering.

中文：我的屏幕在闪烁。 



## 语气转换
写作可以根据预期受众而有所不同。ChatGPT可以产生不同的语气。

In [12]:
prompt = f"""
将以下俚语翻译成商业信函： 
“兄弟，这是乔，看看这个落地灯的规格。”
"""
response = get_completion(prompt)
print(response)

尊敬的先生/女士，我是乔，想请您查看一下这个落地灯的规格。


## 格式转换
ChatGPT可以在不同格式之间进行翻译。提示应该描述输入和输出格式。

In [13]:
data_json = { "餐厅员工" :[ 
    {"姓名":"Shyam", "电子邮件":"shyamjaiswal@gmail.com"},
    {"姓名":"Bob", "电子邮件":"bob32@gmail.com"},
    {"姓名":"Jai", "电子邮件":"jai87@gmail.com"}
]}

prompt = f"""
将以下Python字典从JSON转换为带有列标题和标题的HTML表格：{data_json}\
"""
response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html>
<head>
	<title>Restaurant Staff</title>
	<style>
		table, th, td {
			border: 1px solid black;
			border-collapse: collapse;
			padding: 5px;
		}
	</style>
</head>
<body>
	<h1>Restaurant Staff</h1>
	<table>
		<thead>
			<tr>
				<th>Name</th>
				<th>Email</th>
			</tr>
		</thead>
		<tbody>
			{% for staff in restaurant_staff %}
			<tr>
				<td>{{ staff['姓名'] }}</td>
				<td>{{ staff['电子邮件'] }}</td>
			</tr>
			{% endfor %}
		</tbody>
	</table>
</body>
</html>


In [20]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

## 拼写检查/语法检查。

以下是常见语法和拼写问题的示例以及LLM的响应。

为了向LLM发出您想要它校对您的文本的信号，您可以指示模型进行“校对”或“校对和更正”。

In [15]:
text = [ 
  "黑白小狗的女孩有一个球。",  # The girl has a ball.
  "Yolanda有她的笔记本。", # ok
  "这将是一个漫长的一天。汽车需要更换油吗？",  # Homonyms
  "他们的自由走了。他们将带来他们的行李。",  # Homonyms
  "你需要你的笔记本电脑。",  # Homonyms
  "这种药影响我的睡眠能力。你听说过蝴蝶效应吗？", # Homonyms
  "这个短语是为了检查chatGPT的拼写能力"  # spelling
]
for t in text:
    prompt = f"""校对并纠正以下文本，并重写已更正的版本。如果您没有发现任何错误，请说“未发现错误”。\
在文本周围不要使用任何标点符号：
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

未发现错误。
未发现错误。
未发现错误。
未发现错误。 

重写：他们已经离开了，带着行李。
未发现错误。
未发现错误。

重写：无需重写。
未发现错误。


In [23]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter."""
prompt = f"校对并纠正此评论：```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, even adults like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Also, it's a bit small for the price I paid. I think there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter.


In [24]:
from redlines import Redlines

diff = Redlines(text, response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, <span style="color:red;font-weight:700;">even </span>adults <span style="color:red;font-weight:700;text-decoration:line-through;">also </span>like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style="color:red;font-weight:700;text-decoration:line-through;">It's </span><span style="color:red;font-weight:700;">Also, it's </span>a bit small for <span style="color:red;font-weight:700;text-decoration:line-through;">what </span><span style="color:red;font-weight:700;">the price </span>I <span style="color:red;font-weight:700;text-decoration:line-through;">paid for it though. </span><span style="color:red;font-weight:700;">paid. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before <span style="color:red;font-weight:700;text-decoration:line-through;">I gave </span><span style="color:red;font-weight:700;">giving </span>it to my daughter.

In [25]:
prompt = f"""
校对并纠正此评论。使其更具吸引力。确保它遵循APA样式指南并针对高级读者。\
以markdown格式输出。\
文本：```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

# 评论：可爱的熊猫玩具

我为女儿的生日买了这个熊猫玩具，因为她总是从我的房间里拿走我的。是的，成年人也喜欢熊猫。她随身携带它，它非常柔软和可爱。其中一只耳朵比另一只低，我不认为这是设计上的不对称。但是，相对于价格来说，它有点小了。我认为可能有其他同样价格但更大的选择。它比预期提前了一天到达，所以我在把它送给女儿之前自己玩了一下。

这个评论遵循APA样式指南，使用markdown格式输出。它针对高级读者，更具吸引力，语言更加流畅。